<a href="https://colab.research.google.com/github/errorloadingusername123/Learn_pytorch/blob/main/NeuralNetwork%26CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [290]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
import torchvision.datasets as datasets
import torch.nn.functional as F
import torchvision.transforms as transforms

In [291]:
               #NORMAL NEURAL NETWORK
# class NN(nn.Module):
#   def __init__(self, input_size, num_classes):
#     super(NN, self).__init__()
#     self.fc1 = nn.Linear(input_size, 50)
#     self.fc2 = nn.Linear(50, num_classes)

#   def forward(self, x):
#     x = F.relu(self.fc1(x))
#     # print(x.size())
#     x = self.fc2(x)
#     # print(x.size())
#     return x

In [292]:
              #CREATE SIMPLE CNN
class CNN(nn.Module):
  def __init__(self, input_size = 1, num_classes = 10):
    super(CNN, self).__init__()
    self.conv1 = nn.Conv2d(in_channels = 1, out_channels = 8, kernel_size = 3, stride = 1, padding = 1) #same convolution
    self.pool = nn.MaxPool2d(kernel_size = 2, stride = 2)
    self.conv2 = nn.Conv2d(in_channels = 8, out_channels = 16, kernel_size = 3, stride = 1, padding = 1)
    self.fc1 = nn.Linear(16*7*7,out_features = num_classes)

  def forward(self, x): #Input Shape: [batch_size=1, channels=1, height=28, width=28]
    x = F.relu(self.conv1(x))# (1,1,28,28) -> (1,8,28,28)
    x = self.pool(x) #(1,8,28,28) -> (1,8,14,14)
    x = F.relu(self.conv2(x)) # (1,8,14,14) -> (1,16,14,14)
    x = self.pool(x) # (1,16,14,14) -> (1,16,7,7)
    x = x.reshape(x.shape[0], -1) #flattens to [batch_size, features] (1,16,7,7) -> (1,786)
    x = self.fc1(x) #fully connected layer mapping to 10 outputs (1,16*7*7) -> (1,10)
    return x


In [293]:
#model.state_dict()

In [294]:
model = CNN()
x = torch.randn(64, 1, 28, 28)
print(model(x).shape)


torch.Size([64, 10])


In [295]:
           #Testing conv2d layer
           #1: Weight times input data
           #2: one colour channel image into 8 colour images.
images = torch.rand(32, 1, 28, 28)
test_image = images[0]

conv_layer1 = nn.Conv2d(in_channels = 1, #greyscale black-white
                       out_channels = 8,
                       kernel_size = 3,
                       stride = 1,
                       padding = 1)
conv_output = conv_layer1(test_image)
conv_output.shape

torch.Size([8, 28, 28])

In [296]:
            #Testing maxpool2d layer
            #1: Max of some pixels
images = torch.rand(32, 1, 28, 28)
test_image = images[0]

max_pool_layer = nn.MaxPool2d(kernel_size = 2, stride = 2)
pool_output = max_pool_layer(test_image)
pool_output.shape

torch.Size([1, 14, 14])

In [297]:
# Print out original image shape without and with unsqueezed dimension
print(f"Test image original shape: {test_image.shape}")
print(f"Test image with unsqueezed dimension: {test_image.unsqueeze(dim=0).shape}")

# Create a sample nn.MaxPoo2d() layer

# Pass data through just the conv_layer
test_image_through_conv = conv_layer(test_image)
print(f"Shape after going through conv_layer(): {test_image_through_conv.shape}")

# Pass data through the max pool layer
test_image_through_conv_and_max_pool = max_pool_layer(test_image_through_conv)
print(f"Shape after going through conv_layer() and max_pool_layer(): {test_image_through_conv_and_max_pool.shape}")

Test image original shape: torch.Size([1, 28, 28])
Test image with unsqueezed dimension: torch.Size([1, 1, 28, 28])
Shape after going through conv_layer(): torch.Size([8, 28, 28])
Shape after going through conv_layer() and max_pool_layer(): torch.Size([8, 14, 14])


In [298]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


In [299]:
input_size = 784
num_classes = 10
learning_rate = 0.001
batch_size = 32
num_epochs = 4

In [300]:
train_dataset = datasets.MNIST(root='dataset/', train=True, transform=transforms.ToTensor(), download=True)
train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)

test_dataset = datasets.MNIST(root='dataset/', train=False, transform=transforms.ToTensor(), download=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=True)

In [301]:
model = NN(input_size=input_size, num_classes=num_classes).to(device)

In [302]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [303]:
for epoch in range(num_epochs):
  for data, targets in train_loader:
    data = data.to(device=device)
    targets = targets.to(device=device)

    data = data.reshape(data.shape[0], -1)


    scores = model(data)
    loss = criterion(scores, targets)

    optimizer.zero_grad()
    loss.backward()

    optimizer.step()

In [304]:
def check_accuracy(loader, model):
  if loader.dataset.train:
    print("Checking accuracy on training data")
  else:
    print("Checking accuracy on test data")
  num_correct = 0
  num_samples = 0
  model.eval()

  with torch.no_grad():
    for x, y in loader:
      x = x.to(device = device)
      x.size()
      y = y.to(device = device) #64x1  matrix
      x = x.reshape(x.shape[0], -1)

      scores = model(x) #64 x 10 matrix
      _, predictions = scores.max(1)
      num_correct += (predictions == y).sum()
      num_samples += predictions.size(0)

    print(f'Got {num_correct} / {num_samples} with accuracy {float(num_correct)/float(num_samples)*100:.2f}')

  model.train()
check_accuracy(train_loader, model)
check_accuracy(test_loader, model)

Checking accuracy on training data
Got 58450 / 60000 with accuracy 97.42
Checking accuracy on test data
Got 9657 / 10000 with accuracy 96.57
